In [1]:
from code_tokenizer import *

code = """
a = 1
if(a < 0)
{
    a = 0
}
"""
print(color_by_char_type(code))
parsed = parse(code)
def read(parsed):
    meaning = []
    if(type(parsed[0]) != list):
        parsed = [parsed]
    for item in parsed:
        if(item[0] == 'assign'):
            print(item[1])
read(parsed)
print()
for item in parsed:
    print(item)


a = 1
if(a < 0)
{
    a = 0
}

['a', ['number', '1']]

['assign', ['a', ['number', '1']]]
['if', ['operation', '<', [['word', 'a'], ['number', '0']]], [['assign', ['a', ['number', '0']]]]]
